In [1]:
pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/e2/8c/0f994c6d0c362c2635e1478e175570a9e78d60f19967eb8bf39d2e0bcfab/streamlit-1.40.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/9b/52/4a86a4fa1cc2aae79137cc9510b7080c3e5aede2310d14fae5486feec7f7/altair-5.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/bb/2a/10164ed1f31196a2f7f3799368a821765c62851ead0e630ab52b8e14b4d0/blinker-1.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a4/07/14f8ad37f2d12a5ce41206c21820d8cb6561b728e51fad4530dff0552a67/cachetools-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.19,<4,>=3.0.7 from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc4

In [4]:
!pip install langchain langchain-core langgraph openai typing-extensions langchain_anthropic langchain-core langchain-community selenium scrapy   

In [5]:
pip install pdf2image

Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip install prettytable

  Obtaining dependency information for prettytable from https://files.pythonhosted.org/packages/73/19/4bb9530512432774fdd7cb7c020851d4decbb811d95f86fd4f6a870a6d3e/prettytable-3.12.0-py3-none-any.whl.metadata


In [1]:
import os

# Directly set API keys (replace "your_key_here" with the actual key strings)
os.environ["OPENAI_API_KEY"] = 
os.environ["BRAVESEARCH_API_KEY"] =
os.environ["ANTHROPIC_API_KEY"] = 

In [2]:
#for model architcture
from typing import Literal, TypedDict
from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import logging
import pandas as pd

#for usecase 1
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options
from IPython.display import Image, display, HTML

#for usecase 2
import re
import requests
from bs4 import BeautifulSoup
import pdfplumber
import io

In [3]:
#TOOLS
@tool
def webstaurantstore(query: str = None, product_limit: int = 5) -> str:
    """
    Scrapes the WebstaurantStore website for product data based on the search term with a user-defined product limit.

    Args:
    - query (str): The search term used to find products.
    - product_limit (int): The maximum number of products to retrieve. Defaults to 5.

    Returns:
    - str: A string representation of the products found, including price, URL and description.
    """
    # Setup Edge WebDriver for Web Scraping
    options = Options()
    options.use_chromium = True
    service = EdgeService(EdgeChromiumDriverManager().install())
    
    # Start Edge WebDriver
    driver = webdriver.Edge(service=service, options=options)
    url = f"https://www.webstaurantstore.com/search/{query.replace(' ', '-')}.html"
    driver.get(url)
    
    # Wait for the product containers to load
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-box-container')))
        
        products = []
        product_elements = driver.find_elements(By.CLASS_NAME, 'product-box-container')
        
        for item in product_elements:
            try:
                # Extract product details
                description = item.find_element(By.CSS_SELECTOR, 'span[data-testid="itemDescription"]').text.strip()
                price = item.find_element(By.CSS_SELECTOR, '[data-testid="price"]').text.strip()
                product_link = item.find_element(By.CSS_SELECTOR, "a[data-testid='itemLink']").get_attribute('href')
                
                # Compile product details into a dictionary
                product = {
                    'description': description,
                    'price': price,
                    'product_link': product_link
                }
                
                products.append(product)
                
                if len(products) >= product_limit:  # Limit based on user query or default 5
                    break
            except Exception as e:
                print(f"Error extracting details from item: {e}")
    
    finally:
        driver.quit()  # Ensure the driver is closed
    
    # Return structured product data
    return str(products)


@tool
def icecastlefh(vehicle_name: str = None) -> str:
    """
    Fetches detailed floor plan information from the IceCastleFH website based on the provided vehicle name by extracting data from the associated PDF.

    Args:
    - vehicle_name (str): The name of the vehicle to search for.

    Returns:
    - str: Extracted text content from the floor plan PDF if found; otherwise, an error message or indication that no text was available.
    """
# Helper function to create a URL-friendly slug
    def create_slug(product_name):
        product_name = product_name.lower()
        product_name = product_name.replace('.', '-')
        product_name = re.sub(r"′\s*x\s*", '-x-', product_name)
        product_name = product_name.replace("'", '')
        product_name = re.sub(r'\s+x\s+', 'x', product_name)
        product_name = product_name.replace(' ', '-')
        product_name = re.sub(r'[^a-z0-9\-]', '', product_name)
        product_name = re.sub(r'-{2,}', '-', product_name)
        return product_name.strip('-')

    try:
        # Step 1: Create product slug
        slug = create_slug(vehicle_name)
        product_url = f"https://icecastlefh.com/product/{slug}"
        print(f"Fetching data from: {product_url}")

        # Step 2: Fetch product page
        response = requests.get(product_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Step 3: Find the PDF link
        pdf_link_tag = soup.find('a', href=lambda href: href and '/wp-content/uploads/' in href, 
                                 string=lambda text: text and 'Floor Plan' in text)
        
        if not pdf_link_tag or 'href' not in pdf_link_tag.attrs:
            return "PDF link not found on the product page."

        href = pdf_link_tag['href']
        pdf_url = href if href.startswith("http") else f"https://icecastlefh.com{href}"
        print(f"PDF URL found: {pdf_url}")

        # Step 4: Fetch PDF content
        pdf_response = requests.get(pdf_url)
        pdf_response.raise_for_status()

        # Step 5: Extract text from PDF
        with pdfplumber.open(io.BytesIO(pdf_response.content)) as pdf:
            full_text = ""
            for page in pdf.pages:
                full_text += page.extract_text() + "\n\n"

        return full_text.strip() if full_text.strip() else "No text found in the PDF."

    except requests.exceptions.RequestException as e:
        return f"Error fetching data: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"
    
@tool
def cycletrader(query: str = None):
    """
    Fetches data related to motorcycles from CycleTrader based on the provided query.

    Args:
    - query (str, optional): The query term to search for. Defaults to None.

    Returns:
    - str: The output fetched from CycleTrader in string format.
    """
    print("Output from cycletrader")

@tool
def rvtrader(query: str = None):
    """
    Fetches data related to recreational vehicles from RVTrader based on the provided query.

    Args:
    - query (str, optional): The query term to search for. Defaults to None.

    Returns:
    - str: The output fetched from RVTrader in string format.
    """
    print("Output from rvtrader")

@tool
def boattrader(query: str = None):
    """
    Fetches data related to boats from BoatTrader based on the provided query.

    Args:
    - query (str, optional): The query term to search for. Defaults to None.

    Returns:
    - str: The output fetched from BoatTrader in string format.
    """
    print("Output from boattrader")

@tool
def formatter(products: list, query_type: str = "multiple", sort_by: str = None) -> str:
    """
    Formats the product data dynamically into an HTML table.

    Args:
    - products (list): A list of product dictionaries.
    - query_type (str): 'single' for single product query or 'multiple' for multiple products.
    - sort_by (str, optional): Sorting criteria (e.g., 'price', 'released_year').

    Returns:
    - str: HTML content for a table.
    """
    if not products:
        return "<p>No products available</p>"

    # Apply dynamic sorting if a valid field is provided
    if sort_by and all(sort_by in product for product in products):
        try:
            products.sort(key=lambda x: float(x[sort_by].replace('$', '').replace(',', '').replace('/Each', '')) 
                          if isinstance(x[sort_by], str) and x[sort_by][0].isdigit() 
                          else x[sort_by])
        except ValueError:
            products.sort(key=lambda x: x[sort_by])

    # Build the HTML table
    html_table = """
    <table style="border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;">
        <thead>
            <tr style="background-color: #f2f2f2;">
    """
    # Add table headers dynamically from product keys
    for key in products[0].keys():
        html_table += f"<th style='border: 1px solid #ddd; padding: 8px; text-align: left;'>{key.capitalize()}</th>"
    
    html_table += "</tr></thead><tbody>"

    # Add product rows
    for product in products:
        html_table += "<tr>"
        for key, value in product.items():
            html_table += f"<td style='border: 1px solid #ddd; padding: 8px;'>{value}</td>"
        html_table += "</tr>"

    html_table += "</tbody></table>"

    return html_table

In [4]:
tools = [webstaurantstore, icecastlefh, cycletrader, rvtrader, boattrader, formatter]

tool_node = ToolNode(tools)

# Initialize model
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    """
    Ensures the LLM always adheres to the format by using a system message.
    """
    system_message = "You are a helpful assistant that only provides factual product information. Do not include any opinions, summaries, or extra commentary. Only return product data in the requested format."
    
    # Prepend the system message
    messages = [HumanMessage(role="system", content=system_message)] + state['messages']
    
    response = model.invoke(messages)
    
    return {"messages": [response]}


# Define the workflow graph
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
workflow.add_edge(START, "agent")

# Define the conditional edges based on tool calls
workflow.add_conditional_edges("agent", should_continue)

# Add normal edge from tools to agent
workflow.add_edge("tools", "agent")

# Initialize memory to persist state
checkpointer = MemorySaver()

# Compile the graph into a LangChain Runnable
app = workflow.compile(checkpointer=checkpointer)


In [5]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content='standard options for "6.5′ x 8′ Grandpa’s Hideout"')]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

Fetching data from: https://icecastlefh.com/product/6-5-x-8-grandpas-hideout
PDF URL found: https://icecastlefh.com/wp-content/uploads/2023/12/6.5-x-10-Grandpas_Hideout_2023-1.pdf


'Based on the information retrieved from IceCastleFH, here are the standard options for the "6.5′ x 8′ Grandpa\'s Hideout" ice fishing house:\n\n1. Single Axle Frame\n2. Hand Crank Winch System\n3. 4 Holes\n4. Factory Select Paneling\n5. Cedar Trim\n6. Flat Ceiling\n7. Carpet\n8. LED Package\n9. 12V Receptacle\n10. 110V Receptacles\n11. 6\' Upper cabinet\n12. Flat Panel Doors with New Hinges\n13. 20,000 BTU Furnace\n14. 68" Jack-knife sofa\n15. New Countertops\n16. Tank Light\n17. EXT 110V\n18. Converter\n19. 12v Switches\n\nPlease note that the information provided is based on the data retrieved from the IceCastleFH system, which appears to be for a slightly different model (6.5 X 10V Grandpa\'s). The options listed are those that were clearly identifiable from the retrieved data.'

In [6]:
from IPython.display import display, HTML

def run_chatbot():
    """
    A chatbot implementation that displays the formatted output in a Jupyter environment.
    """
    print("Welcome to CCC - AI Product Assistant! Type 'exit' to end the chat.")
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("Goodbye!")
            break
        
        try:
            # Pass user input as a message to the agent
            final_state = app.invoke(
                {"messages": [HumanMessage(content=user_input)]},
                config={"configurable": {"thread_id": 42}}
            )
            
            # Extract response
            response = final_state["messages"][-1].content
            
            # Detect if the response contains HTML for table formatting
            if "<table" in response:
                print("\nAI Assistant (Table):")
                display(HTML(response))  # Render HTML table in Jupyter
            else:
                print(f"\nAI Assistant: {response}")
        
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the chatbot
run_chatbot()

Welcome to CCC - AI Product Assistant! Type 'exit' to end the chat.

You: top 3 two door fridges

AI Assistant (Table):


Description,Price,Product_link
"Avantco GDS-47-HC 53"" Black Customizable Sliding Glass Door Merchandiser Refrigerator with LED Lighting","$1,779.00/Each",https://www.webstaurantstore.com/avantco-gds-47-hc-53-black-sliding-glass-door-merchandiser-refrigerator-with-led-lighting/178GDS47HCB.html
"Avantco GDW-49-BB 53"" Black Customizable Two Door Wine Merchandiser with Black Customizable Interior","$2,019.00/Each",https://www.webstaurantstore.com/avantco-gdw-49-bb-53-black-two-door-wine-merchandiser-with-black-interior/178GDW49BB.html
"Avantco GDC-40-HC 48"" Black Customizable Swing Glass Door Merchandiser Refrigerator with LED Lighting","$1,699.00/Each",https://www.webstaurantstore.com/avantco-gdc-40-hc-48-black-swing-glass-door-merchandiser-refrigerator-with-led-lighting/178GDC40HCB.html



You: exit
Goodbye!


In [18]:
final_state = app.invoke(
    {"messages": [HumanMessage(content='price of Solwave Ameri-Series Medium-Duty Stainless Steel Commercial Microwave with Push Button Controls - 120V, 1,200W')]},
    config={"configurable": {"thread_id": 42}}
)

formatted_table = final_state["messages"][-1].content
display(HTML(formatted_table))

In [19]:
final_state = app.invoke(
    {"messages": [HumanMessage(content='price of Main Street Equipment BMR-49-R 54" Solid Door Reach-In Refrigerator')]},
    config={"configurable": {"thread_id": 42}}
)

formatted_table = final_state["messages"][-1].content
display(HTML(formatted_table))

In [ ]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content='standard options for "6.5′ x 8′ Grandpa’s Hideout"')]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

In [7]:
from IPython.display import display, HTML

def run_chatbot():
    """
    A chatbot implementation that displays the formatted output in a Jupyter environment.
    """
    print("Welcome to CCC - AI Product Assistant! Type 'exit' to end the chat.")
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("Goodbye!")
            break
        
        try:
            # Pass user input as a message to the agent
            final_state = app.invoke(
                {"messages": [HumanMessage(content=user_input)]},
                config={"configurable": {"thread_id": 42}}
            )
            
            # Extract response
            response = final_state["messages"][-1].content
            
            # Detect if the response contains HTML for table formatting
            if "<table" in response:
                print("\nAI Assistant (Table):")
                display(HTML(response))  # Render HTML table in Jupyter
            else:
                print(f"\nAI Assistant: {response}")
        
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the chatbot
run_chatbot()

Welcome to CCC - AI Product Assistant! Type 'exit' to end the chat.

You: give me top 3 results for reach-in freezer

AI Assistant (Table):


Description,Price,Product_link
"Avantco A-49F-HC 54"" Solid Door Reach-In Freezer","$2,249.00/Each",https://www.webstaurantstore.com/avantco-a-49f-hc-54-solid-door-reach-in-freezer/178A49FHC.html
"Avantco A-19F-HC 29"" Solid Door Reach-In Freezer","$1,449.00/Each",https://www.webstaurantstore.com/avantco-a-19f-hc-29-solid-door-reach-in-freezer/178A19FHC.html
"Avantco SS-3F-HC 81 5/16"" Stainless Steel Solid Door Reach-In Freezer","$4,069.00/Each",https://www.webstaurantstore.com/avantco-ss-3f-hc-80-7-8-solid-door-reach-in-freezer/178SS3FHC.html



You: give me the 3 most expensive reach-in freezers
Error extracting details from item: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[data-testid="price"]"}
  (Session info: MicrosoftEdge=130.0.2849.80); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D346DCA5+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7D3718384+2250164]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF7D36576D6+1460486]
	(No symbol) [0x00007FF7D32697CC]
	(No symbol) [0x00007FF7D326990C]
	(No symbol) [0x00007FF7D326066C]
	(No symbol) [0x00007FF7D328853F]
	(No symbol) [0x00007FF7D3260617]
	(No symbol) [0x00007FF7D32604DD]
	(No symbol) [0x00007FF7D32887E0]
	(No symbol) [0x00007FF7D3260617]
	(No symbol) [0x00007FF7D32A1431]
	(No symbol) [0x00007FF7D3288163]
	(No symbol) [0x00007FF7D325FB54]
	(No s

Description,Price,Product_link
"True T-49FG-HC~FGD01 54 1/8"" 2 Section Glass Door Reach-In Freezer with LED Lighting","$9,602.00/Each",https://www.webstaurantstore.com/true-t49fghcfgd01-54-2-section-glass-door-reachin-freezer-with-led-lighting/200T49FGHC.html
"True TS-72F-HC 78 1/8"" Stainless Steel Solid Door Reach-In Freezer","$9,430.00/Each",https://www.webstaurantstore.com/true-ts-72f-hc-78-stainless-steel-solid-door-reach-in-freezer/200TS72FHC.html
"True T-72F-HC 78 1/8"" Solid Door Reach-In Freezer","$8,316.00/Each",https://www.webstaurantstore.com/true-t-72f-hc-78-1-8-three-section-solid-door-reach-in-freezer/200T72FHC.html



You: exit
Goodbye!
